In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

In [8]:
# function for appending data in json format
def write_json(new_data, filename='data.json'):
    with open(filename, 'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with file_data inside emp_details
        file_data["comments"].append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent=4)

In [10]:
# installing chrome driver
driver = webdriver.Chrome(ChromeDriverManager().install())
# takes to our desired page
driver.get("https://www.youtube.com/watch?v=bpQFCcSI0pU")
wait = WebDriverWait(driver,20)

In [11]:
# a dictionary that holds all the comments
comments_dict = {
    "comments":[]
}
# creating a json file to store data
with open("data.json", "w") as f:
    json.dump(comments_dict, f, indent=2)

In [12]:
time.sleep(2)
# ytd_comments = driver.find_elements_by_css_selector("div#contents>ytd-comment-thread-renderer")
# print(len(ytd_comments))
driver.execute_script("window.scrollBy(0,700)", "")
time.sleep(2)
ytd_comments = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div#contents>ytd-comment-thread-renderer")))
print(len(ytd_comments))

20


In [14]:
comment = 0
comment_index = 0
num_of_reply = 0
slicing_num = 0
while True:
    # finding the comments
    ytd_comments = driver.find_elements_by_css_selector("div#contents>ytd-comment-thread-renderer")
    # length of the list of comments
    old_ytd_comments_len = len(ytd_comments)
    print(old_ytd_comments_len)
    # this section rejects previous comments and only take the comments that aren't in the save file(data.json)
    sliced_ytd_comments = ytd_comments[slicing_num:]
    # for extracting comments and replies
    for ytd_comment in sliced_ytd_comments:
        # this section shows the comment in the screen
        driver.execute_script("arguments[0].scrollIntoView();",ytd_comment)
        # extracting the main comment
        main_comment = ytd_comment.find_element_by_id("content")
        main_comment_text = main_comment.text
        print(main_comment_text)
        # all the replies will be stored in this list
        replies_list = []
        try:
            time.sleep(.5)
            # finding replies button and clicking it so that we can extract the value
            replies_button = ytd_comment.find_element_by_css_selector("ytd-button-renderer#more-replies")
            driver.execute_script("arguments[0].click();", replies_button) 
            time.sleep(1)
            while True:
                # a protion of replies that is rendered by javascript
                replies = ytd_comment.find_elements_by_css_selector("yt-formatted-string#content-text")
                # checking how many replies youtube shows at a time
                old_reply_len = len(replies)
                # this captures how many replies we stored
                reply_index = 0
                print(len(replies))
                for reply in replies:
                    # checker for rejecting any reply that is stored in our json file
                    if reply_index == num_of_reply:
                        driver.execute_script("arguments[0].scrollIntoView();",reply)
                        reply_text = reply.text
                        if reply_index != 0:
                            replies_list.append(reply_text)
                        print(reply.text)
                        num_of_reply += 1
                        reply_index += 1
                    else:
                        reply_index +=1
                time.sleep(.5)
                # replies that weren't loaded first time, this button loads them
                see_more_replies_button = ytd_comment.find_element_by_css_selector("div#button>ytd-button-renderer")
                #time.sleep(2)
                # clicking the see more replies button
                driver.execute_script("arguments[0].click();", see_more_replies_button) 
                time.sleep(1.5)
                # we again find how many replies we have
                replies = ytd_comment.find_elements_by_css_selector("yt-formatted-string#content-text")
                print(len(replies))
                new_reply_len = len(replies)
                # if we don't have any new replies then replies loop breakes and goes to next comment
                if old_reply_len == new_reply_len:
                    
                    num_of_reply = 0
                    break
                else:
                    pass
        except:
            num_of_reply = 0
        # this dict stores the main comment and all the replies
        single_comment_dict = {
            "main_comment":main_comment_text,
            "replies":replies_list
            
        }
        # to store data we are calling the function
        write_json(single_comment_dict)
        # this stores how many comments we stored as we don't want to store the same data more than once
        slicing_num += 1
        print()
        print()
        print()
    time.sleep(2)
    # after scrolling the page new comments arrive and this portion detects how many new comments came 
    ytd_comments = driver.find_elements_by_css_selector("div#contents>ytd-comment-thread-renderer")
    new_ytd_comments_len = len(ytd_comments)
    # if there is no new comments then the programme finishes
    if old_ytd_comments_len == new_ytd_comments_len:
        break
driver.quit()     

AttributeError: 'WebDriver' object has no attribute 'find_elements_by_css_selector'